In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
path = r'C:\Users\Treeb\Instacart Basket Analysis 11-20-2023'

In [3]:
df_ords_prods_merge = pd.read_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'orders_products_merged_plus_cols.pkl'))

In [5]:
df_ords_prods_merge.shape

(32404859, 18)

In [4]:
df_ords_prods_merge.groupby('department_id').agg({'order_number': ['mean']})

order_number
                      mean
department_id             
1                15.457838
2                17.277920
3                17.170395
4                17.811403
5                15.215751
6                16.439806
7                17.225802
8                15.340650
9                15.895474
10               20.197148
11               16.170638
12               15.887671
13               16.583536
14               16.773669
15               16.165037
16               17.665606
17               15.694469
18               19.310397
19               17.177343
20               16.473447
21               22.902379

The results of aggregating the mean of the order number grouped by department id for the entire dataframe are above. The results from the subset of this dataframe are shown below:  

    order_number
mean
department_id	
4	18.825780
7	17.472355
13	17.993423
14	19.246334
16	19.463012
17	11.294069
19	19.305237
20	17.599636

First thing we notice is that there are less total department_id values. It seems not all of these departments were in the first one million rows of data. Next thing we notice is that the mean values are different from each other. Most of them differ by only 1 or less than 1, but some --for example department_id = 17-- differ by more than 4. 1 million rows of data is very different from over 32 million rows. 

### Create Loyalty Flag

#### Create max_order column

In [8]:
df_ords_prods_merge['max_order'] = df_ords_prods_merge.groupby(['user_id'])['order_number'].transform(np.max)

#### Criteria for 'Loyal Customer'

In [9]:
df_ords_prods_merge.loc[df_ords_prods_merge['max_order'] > 40, 'loyalty_flag'] = 'Loyal customer'

#### Criteria for 'Regular Customer'

In [10]:
df_ords_prods_merge.loc[(df_ords_prods_merge['max_order'] <= 40) & (df_ords_prods_merge['max_order'] > 10), 'loyalty_flag'] = 'Regular customer'

#### Criteria for 'New Customer'

In [11]:
df_ords_prods_merge.loc[df_ords_prods_merge['max_order'] <= 10, 'loyalty_flag'] = 'New customer'

#### Value counts of values in 'loyalty_flag' column

In [12]:
df_ords_prods_merge['loyalty_flag'].value_counts(dropna = False)

Regular customer    15876776
Loyal customer      10284093
New customer         6243990
Name: loyalty_flag, dtype: int64

In [13]:
df_ords_prods_merge.head()

order_id  user_id  order_number  orders_day_of_week  order_hour_of_day  \
0   2539329        1             1                   2                  8   
1   2398795        1             2                   3                  7   
2    473747        1             3                   3                 12   
3   2254736        1             4                   4                  7   
4    431534        1             5                   4                 15   

   days_since_users_last_order  product_id  add_to_cart_order  reordered  \
0                          NaN         196                  1          0   
1                         15.0         196                  1          1   
2                         21.0         196                  1          1   
3                         29.0         196                  1          1   
4                         28.0         196                  1          1   

  _merge product_name  aisle_id  department_id  prices    price_range_loc  \
0   both         Soda        77              7     9.0  Mid-range product   
1   both         Soda        77              7     9.0  Mid-range product   
2   both         Soda        77              7     9.0  Mid-range product   
3   both         Soda        77              7     9.0  Mid-range product   
4   both         Soda        77              7     9.0  Mid-range product   

      busiest_day    busiest_days busiest_period_of_day  max_order  \
0  Regularly busy  Regularly busy        Average orders         10   
1  Regularly busy        Slow day        Average orders         10   
2  Regularly busy        Slow day           Most orders         10   
3      Least busy        Slow day        Average orders         10   
4      Least busy        Slow day           Most orders         10   

   loyalty_flag  
0  New customer  
1  New customer  
2  New customer  
3  New customer  
4  New customer

### Average prices for products purchased Grouped by customer loyalty

In [14]:
df_ords_prods_merge.groupby('loyalty_flag').agg({'prices': ['mean']})

prices
                       mean
loyalty_flag               
Loyal customer    10.386336
New customer      13.294670
Regular customer  12.495717

### Low vs. High Spenders

In [15]:
df_ords_prods_merge['average_purchase_price'] = df_ords_prods_merge.groupby(['user_id'])['prices'].transform(np.average)

In [16]:
df_ords_prods_merge.head()

order_id  user_id  order_number  orders_day_of_week  order_hour_of_day  \
0   2539329        1             1                   2                  8   
1   2398795        1             2                   3                  7   
2    473747        1             3                   3                 12   
3   2254736        1             4                   4                  7   
4    431534        1             5                   4                 15   

   days_since_users_last_order  product_id  add_to_cart_order  reordered  \
0                          NaN         196                  1          0   
1                         15.0         196                  1          1   
2                         21.0         196                  1          1   
3                         29.0         196                  1          1   
4                         28.0         196                  1          1   

  _merge  ... aisle_id  department_id  prices    price_range_loc  \
0   both  ...       77              7     9.0  Mid-range product   
1   both  ...       77              7     9.0  Mid-range product   
2   both  ...       77              7     9.0  Mid-range product   
3   both  ...       77              7     9.0  Mid-range product   
4   both  ...       77              7     9.0  Mid-range product   

      busiest_day    busiest_days busiest_period_of_day max_order  \
0  Regularly busy  Regularly busy        Average orders        10   
1  Regularly busy        Slow day        Average orders        10   
2  Regularly busy        Slow day           Most orders        10   
3      Least busy        Slow day        Average orders        10   
4      Least busy        Slow day           Most orders        10   

   loyalty_flag average_purchase_price  
0  New customer               6.367797  
1  New customer               6.367797  
2  New customer               6.367797  
3  New customer               6.367797  
4  New customer               6.367797  

[5 rows x 21 columns]

In [17]:
df_ords_prods_merge.loc[df_ords_prods_merge['average_purchase_price'] >= 10, 'spending_flag'] = 'High Spender'

In [18]:
df_ords_prods_merge.loc[df_ords_prods_merge['average_purchase_price'] < 10, 'spending_flag'] = 'Low Spender'

In [19]:
df_ords_prods_merge['spending_flag'].value_counts(dropna = False)

Low Spender     31770646
High Spender      634213
Name: spending_flag, dtype: int64

### Frequent Vs. Non-Frequent Customers

In [20]:
df_ords_prods_merge['median_days_between_orders'] = df_ords_prods_merge.groupby(['user_id'])['days_since_users_last_order'].transform(np.median)

In [21]:
df_ords_prods_merge.loc[df_ords_prods_merge['median_days_between_orders'] > 20, 'frequency_flag'] = 'Non-Frequent customer'

In [22]:
df_ords_prods_merge.loc[(df_ords_prods_merge['median_days_between_orders'] <= 20) & (df_ords_prods_merge['median_days_between_orders'] > 10), 'frequency_flag'] = 'Regular customer'

In [23]:
df_ords_prods_merge.loc[df_ords_prods_merge['median_days_between_orders'] <= 10, 'frequency_flag'] = 'Frequent customer'

In [24]:
df_ords_prods_merge['frequency_flag'].value_counts(dropna = False)

Frequent customer        21559853
Regular customer          7208564
Non-Frequent customer     3636437
NaN                             5
Name: frequency_flag, dtype: int64

In [25]:
df_ords_prods_merge.head()

order_id  user_id  order_number  orders_day_of_week  order_hour_of_day  \
0   2539329        1             1                   2                  8   
1   2398795        1             2                   3                  7   
2    473747        1             3                   3                 12   
3   2254736        1             4                   4                  7   
4    431534        1             5                   4                 15   

   days_since_users_last_order  product_id  add_to_cart_order  reordered  \
0                          NaN         196                  1          0   
1                         15.0         196                  1          1   
2                         21.0         196                  1          1   
3                         29.0         196                  1          1   
4                         28.0         196                  1          1   

  _merge  ...    price_range_loc     busiest_day    busiest_days  \
0   both  ...  Mid-range product  Regularly busy  Regularly busy   
1   both  ...  Mid-range product  Regularly busy        Slow day   
2   both  ...  Mid-range product  Regularly busy        Slow day   
3   both  ...  Mid-range product      Least busy        Slow day   
4   both  ...  Mid-range product      Least busy        Slow day   

   busiest_period_of_day max_order  loyalty_flag average_purchase_price  \
0         Average orders        10  New customer               6.367797   
1         Average orders        10  New customer               6.367797   
2            Most orders        10  New customer               6.367797   
3         Average orders        10  New customer               6.367797   
4            Most orders        10  New customer               6.367797   

  spending_flag  median_days_between_orders         frequency_flag  
0   Low Spender                        20.5  Non-Frequent customer  
1   Low Spender                        20.5  Non-Frequent customer  
2   Low Spender                        20.5  Non-Frequent customer  
3   Low Spender                        20.5  Non-Frequent customer  
4   Low Spender                        20.5  Non-Frequent customer  

[5 rows x 24 columns]

### Export Final Dataframe as Pickle

In [26]:
df_ords_prods_merge.to_pickle(os.path.join(path, '02 Data','Prepared Data', 'orders_products_merged_flags.pkl'))